In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git

In [ ]:
!pip uninstall -y jax
!pip install -e .[bitsandbytes,liger-kernel]
!pip install pandas
!pip install huggingface_hub

In [ ]:
# Лучше не трогать
%cd LLaMA-Factory

In [ ]:
!pip install -r requirements.txt
!pip install llamafactory

In [ ]:
%cd ..

In [ ]:
base_model = "Qwen/Qwen2.5-32B-Instruct"

In [ ]:
from huggingface_hub import login, snapshot_download

login("hf_token")

snapshot_download(base_model)

In [ ]:
!pwd

In [ ]:
!mc ls minio/workspace/models/lora/Qwen/Qwen2.5-32B-Instruct/matching/

In [ ]:
lora_path = f"minio/workspace/models/lora/Qwen/Qwen2.5-32B-Instruct/matching/train_2025-01-28-10:30:29"

In [ ]:
lora_output_path = f"/home/jupyter/lora/{'/'.join(lora_path.split('/')[-4:])}"

In [ ]:
lora_output_path

In [ ]:
!ls  /home/jupyter/lora/Qwen/Qwen2.5-32B-Instruct/matching/train_2025-01-28-10:30:29

In [ ]:
command = f"""
mc cp \
{lora_path} \
{'/'.join(lora_output_path.split('/')[:-1])} --recursive
"""

print(command)

get_ipython().system(command)

In [ ]:
command = f"""

llamafactory-cli export \
  --model_name_or_path {base_model} \
  --adapter_name_or_path {lora_output_path} \
  --template qwen \
  --finetuning_type lora \
  --export_dir /home/jupyter/merge_model \
  --export_size 2 \
  --export_device cpu \
  --export_legacy_format false
"""

print(command)

get_ipython().system(command)


In [ ]:
!rm -Rf AutoFP8
!git clone --depth 1 https://github.com/neuralmagic/AutoFP8.git
!pip install -e AutoFP8

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from auto_fp8 import AutoFP8ForCausalLM, BaseQuantizeConfig

pretrained_model_dir = "/home/jupyter/merge_model"
quantized_model_dir = "/home/jupyter/merge_model_fp8"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_dir, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Load and tokenize 512 dataset samples for calibration of activation scales
ds = load_dataset("mgoin/ultrachat_2k", split="train_sft").select(range(512))
examples = [tokenizer.apply_chat_template(batch["messages"], tokenize=False) for batch in ds]
examples = tokenizer(examples, padding=True, truncation=True, return_tensors="pt").to("cuda")

# Define quantization config with static activation scales
quantize_config = BaseQuantizeConfig(quant_method="fp8", activation_scheme="static")

# Load the model, quantize, and save checkpoint
model = AutoFP8ForCausalLM.from_pretrained(pretrained_model_dir, quantize_config)
model.quantize(examples)
model.save_quantized(quantized_model_dir)


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from auto_fp8 import AutoFP8ForCausalLM, BaseQuantizeConfig

pretrained_model_dir = "/home/jupyter/merge_model"
quantized_model_dir = "/home/jupyter/merge_model_fp8"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_dir, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

df = pd.read_csv("train_data.csv").head(2000)

def format_example(row):
    return f"User: {row['input']}\nAssistant: {row['output']}"

examples_texts = df.apply(format_example, axis=1).tolist()

ds = Dataset.from_dict({"text": examples_texts})

examples = tokenizer(ds["text"], padding=True, truncation=True, return_tensors="pt").to("cuda")

quantize_config = BaseQuantizeConfig(quant_method="fp8", activation_scheme="static")

model = AutoFP8ForCausalLM.from_pretrained(pretrained_model_dir, quantize_config)
model.quantize(examples)
model.save_quantized(quantized_model_dir)


In [ ]:
command = f"""
mc cp {quantized_model_dir} minio/workspace/models/merged_fp8/{'/'.join(lora_path.split('/')[-4:])} --recursive
"""
print(command)
get_ipython().system(command)
